<a href="https://colab.research.google.com/github/CUknot/NLP/blob/main/HW7_beam_seach_to_student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW7: Beam Search Decoding - News Headline Generation

In this exercise, you are going to learn and implement decoding techniques for sequence generation. Usually, the sequence is generated word-by-word from a model. In each step, the model predicted the most likely word based on the predicted words in previous steps (this is called auto-regressive decoding).

As such, it is very important how you decide on what to predicted at each step, as it will be conditioned on to predicted all of the following steps. We will implement two of main decoding techniques introduced in the lecture: **Greedy Decoding** and **Beam Search Decoding**. Greedy Decoding immediately chooses the word with best score at each step, while Beam Search Decoding focuses on the sequence that give the best score overall.

To complete this exercise, you will need to complete the methods for decoding for a text generation model trained on [New York Times Comments and Headlines dataset](https://www.kaggle.com/aashita/nyt-comments). The model is trained to predict a headline for the news given seed text. You do not need to train any model model in this exercise as we provide both the pretrained model and dictionary.


## Download model and vocab and setup

In [ ]:
!wget -O vocab.txt https://www.dropbox.com/s/ht12ua9vpkep6l8/hw9_vocab.txt?dl=0
!wget -O model.bin https://www.dropbox.com/s/okmri7cnd729rr5/hw9_model.bin?dl=0

--2025-02-22 08:55:26--  https://www.dropbox.com/s/ht12ua9vpkep6l8/hw9_vocab.txt?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.66.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.66.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/zlkw3il9cj4c121vtrrxh/hw9_vocab.txt?rlkey=m5gflik1lhvpenwydd3gfwvp7&dl=0 [following]
--2025-02-22 08:55:27--  https://www.dropbox.com/scl/fi/zlkw3il9cj4c121vtrrxh/hw9_vocab.txt?rlkey=m5gflik1lhvpenwydd3gfwvp7&dl=0
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc436af178cef1d8c4b7b3325d0a.dl.dropboxusercontent.com/cd/0/inline/CkkCFfs2sJqbYwIBRoIJkzcBLDilIBleQ7YkbB3W5G0bs7ZuCQkVG60Sf-sf1Bg-VwF5l7uvrTZjAazmQ2RIgH0kwbeg80iJX_hEkc-IMSZr_fHhoZW6MIzh8lC7GVGTCOUuKtlkxjGFVa5mFC_kF2bc/file# [following]
--2025-02-22 08:55:28--  https://uc436af178cef1d8c4b7b3325d0a.dl.dropboxusercont

In [ ]:
import torch
import torch.nn as nn
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Whitespace

In [ ]:
class RNNmodel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, dropout_rate):

        super().__init__()
        self.embedding_dim = embedding_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, 128, num_layers=2,
                     batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(128, vocab_size)

    def forward(self, src):
        embedding = self.embedding(src)
        output,_ = self.rnn(embedding)
        output = self.dropout(output)
        prediction = self.fc2(output)
        return prediction

In [ ]:
with open("vocab.txt") as f:
  vocab_file = f.readlines()
embedding_dim = 64
dropout_rate = 0.2

model = RNNmodel(len(vocab_file), embedding_dim, dropout_rate)
model.load_state_dict(torch.load("model.bin",map_location='cpu'))
model.eval()

<ipython-input-7-e56f35018f72>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.bin",map_location='cpu'))


RNNmodel(
  (embedding): Embedding(10054, 64)
  (rnn): LSTM(64, 128, num_layers=2, batch_first=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=128, out_features=10054, bias=True)
)

In [ ]:
vocab = [v.strip() for v in vocab_file]
vocab_size = len(vocab)
print(f"Vocab Size: {vocab_size}")
vocab[:10]

Vocab Size: 10054


['<unk>', '<pad>', '<eos>', 'the', 'a', 'to', 'of', 's', 'in', 'for']

In [ ]:
stoi = { ch:i for i,ch in enumerate(vocab) }
tokenizer = Tokenizer(WordLevel(stoi, unk_token="<unk>"))
tokenizer.pre_tokenizer = Whitespace()
tokenized_text = tokenizer.encode("the a of to unknowns")
print(tokenized_text)
print(tokenized_text.ids)
print(tokenized_text.tokens)
print(tokenizer.decode(tokenized_text.ids))

Encoding(num_tokens=5, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
[3, 4, 6, 5, 0]
['the', 'a', 'of', 'to', '<unk>']
the a of to <unk>


## 1. TODO: Greedy decode
Normally, in sequence generation task, the model will continue generating tokens until an end-of-sequence symbol appear or the maximum length is reached. For this task:
- The end-of-sequence symbol is "< eos >" and its index is 2
- Use the maximum generation length of 15

In [ ]:
eos_token = '<eos>'
max_gen_length = 15

In [ ]:
import torch.nn.functional as F
def greedy_decode(seed_text, tokenizer):
    """Greedy decodes with seed text.

        Args:
        seed_text: The seed string to be used as initial input to the model.
        tokenizer: The tokenizer for converting word to index and back.

        Your code should do the followings:
          1. Convert current_text to sequences of indices
          2. Predict the next token using the model and choose the token with the highest score as output
          3. Append the predicted index to current_text
          4. Loop until completion
          5. Return text prediction and a list of probabilities of each step

        You do not need to stop early when end-of-sequence token is generated and can continue decoding
        until max_gen_length is reached. We can filter the eos token out later.
    """
    probs = []
    token_ids = tokenizer.encode(seed_text).ids
    current_text = torch.tensor([token_ids]).long()

    with torch.no_grad():
        for _ in range(max_gen_length):
            logits = model(current_text)
            prob = F.softmax(logits[0, -1], dim=-1)
            pred_token_id = torch.argmax(prob).item()
            token_ids.append(pred_token_id)
            current_text = torch.tensor([token_ids]).long()
            probs.append(prob[pred_token_id].item())

    return token_ids, probs

In [ ]:
def clean_output(text, eos_token):
    """Drop eos_token and every word that follows"""
    words = text.split(" ")
    return words.index(eos_token) if eos_token in words else len(words)

In [ ]:
sample_seeds = ["to", "america", "people", "next", "picture", "on"]
for seed in sample_seeds:
    output_tokenID, probs = greedy_decode(seed, tokenizer)
    output_text = tokenizer.decode(output_tokenID)
    idx = clean_output(output_text, eos_token)
    output_text = ' '.join(output_text.split(' ')[:idx])
    probs = probs[:idx-1]
    print(f"output text: {output_text}")
    print(f"probs: {probs}")

output text: to encourage creativity in the new york bill
probs: [0.04071049019694328, 0.26613035798072815, 0.2581591308116913, 0.23116429150104523, 0.07794304192066193, 0.06977992504835129, 0.2244517058134079]
output text: america s lethal export
probs: [0.449653685092926, 0.046399783343076706, 0.7206941843032837]
output text: people to balloon to make a criminal with a dog with a callous rival
probs: [0.3584842383861542, 0.0394657626748085, 0.2695293426513672, 0.033772896975278854, 0.28156647086143494, 0.03089667297899723, 0.32625508308410645, 0.3224399983882904, 0.03489116579294205, 0.22298352420330048, 0.5808276534080505, 0.04674506559967995, 0.26616597175598145]
output text: next phenom english clubs 2 call another deal in the same arrivals
probs: [0.1724235862493515, 0.23235656321048737, 0.7396264672279358, 0.07888385653495789, 0.04404296353459358, 0.05769333988428116, 0.03296951204538345, 0.33040061593055725, 0.6040366291999817, 0.03174585849046707, 0.07468772679567337]
output t

Your output should be:

*   to encourage creativity in the new york bill
*   america s lethal export
*   people to balloon to make a criminal with a dog with a callous rival
*   next phenom english clubs 2 call another deal in the same arrivals
*   picture perfect chapter a spot of view of banning care  
*   on the catwalk in saudi arabia







## 2. TODO: Beam search decode

Another well-known decoding method is beam search decoding that focuses more on the overall sequence score.

Instead of greedily choosing the token with the highest score for each step, beam search decoding expands all possible next tokens and keeps the __k__ most likely sequence at each step, where __k__ is a user-specified beam size. A sequence score is also calculated according user-specified cal_score() function.
The beam with the highest score after the decoding process is done will be the output.

There are a few things that you need to know before implementing a beam search decoder:
- When the eos token is produced, you can stop expanding that beam
- However, the ended beams must be sorted together with active beams
- The decoding ends when every beams are either ended or reached the maximum length, but for this task, you can continue decoding until the max_gen_len is reached
- We usually work with probability in log scale to avoid numerical underflow. You should use np.log(score) before any calculation
- **As probabilities for some classes will be very small, you must add a very small value to the score before taking log e.g np.log(prob + 0.00000001)**

#### Sequence Score
The naive way to calculate the sequence score is to __multiply every token scores__ together. However, doing so will make the decoder prefer shorter sequence as you multiply the sequence score with a value between \[0,1\] for every tokens in the sequence. Thus, we usually normalize the sequence score with its length by calculating its __geometric mean__ instead.

**You should do this in log scale**

In [ ]:
import numpy as np
def cal_score(score_list, length, normalized=False): #cal score for each beam from a list of probs
    total_log_score = np.sum([np.log(i + 1e-8) for i in score_list])
    if normalized:
        return total_log_score / length
    else:
        return total_log_score

In [ ]:
def beam_search_decode(seed_text, max_gen_len, tokenizer, beam_size=5, normalized=False):
    """We will do beam search decoing using seed text in this function.

    Output:
    beams: A list of top k beams after the decoding ended, each beam is a list of
      [seed_text, list of scores, length]

    Your code should do the followings:
    1.Loop until max_gen_len is reached.
    2.During each step, loop thorugh each beam and use it to predict the next word.
      If a beam is already ended, continues without expanding.
    3.Sort all hypotheses according to cal_score().
    4.Keep top k hypotheses to be used at the next step.
    """
    # For each beam we will store (generated text, list of scores, and current length, is_finished)
    seed_ids = torch.tensor(tokenizer.encode(seed_text).ids).unsqueeze(0).long()
    beams = [(seed_ids, [], 0, False)]  # (token_ids, score_list, length, is_finished)

    for _ in range(max_gen_len):
        new_beams = []

        for token_ids, score_list, length, is_finished in beams:
            if is_finished:
                new_beams.append((token_ids, score_list, length, is_finished))
                continue

            logits = model(token_ids)
            probabilities = F.softmax(logits[0, -1], dim=-1)  # Get probability for the last token

            for next_token_id, next_token_prob in enumerate(probabilities):
                new_token_ids = torch.cat((token_ids, torch.tensor([[next_token_id]])), dim=1)
                new_length = length + 1
                new_finished = next_token_id == stoi[eos_token]
                new_score = score_list + [next_token_prob.item()]

                new_beams.append((new_token_ids, new_score, new_length, new_finished))

        # Keep top-k beams based on calculated scores
        new_beams.sort(key=lambda x: cal_score(x[1], x[2], normalized), reverse=True)
        beams = new_beams[:beam_size]

    return beams

## 3. Generate!
Generate 6 sentences based on the given seed texts.

Decode with the provided seed texts with beam_size 5, max_gen_len 10. Compare the results between greedy, normalized, and unnormalized decoding.

Print the result using greedy decoding and top 2 results each using unnormalized and normalized decoing for each seed text.

Also, print scores of each candidate according to cal_score(). Use normalization for greedy decoding.

In [ ]:
def gen_greedy(seed):
  print("-Greedy-")
  output_tokenID, probs = greedy_decode(seed, tokenizer)
  output_text = tokenizer.decode(output_tokenID)
  idx = clean_output(output_text, eos_token)
  output_text = ' '.join(output_text.split(' ')[:idx])
  probs = probs[:idx-1]

  score = cal_score(probs, len(probs), normalized=True)
  print(f"{output_text}  {round(np.exp(score), 2): .2f}")

In [ ]:
def gen_beam_True(seed):
    print("-Normalized-")
    beams = beam_search_decode(seed, max_gen_length, tokenizer, beam_size=5, normalized=True)
    for beam in beams[:2]:
      output_tokenID, probs, length, _ = beam
      output_text = tokenizer.decode(output_tokenID.squeeze(0).tolist())
      idx = clean_output(output_text, eos_token)
      output_text = ' '.join(output_text.split(' ')[:idx])
      probs = probs[:idx-1]

      score = cal_score(probs, len(probs), normalized=True)
      print(f"{output_text}  {round(np.exp(score), 2): .2f}")

In [ ]:
def gen_beam_False(seed):
    print("-Unnormalized-")
    beams = beam_search_decode(seed, max_gen_length, tokenizer, beam_size=5, normalized=False)
    for beam in beams[:2]:
      output_tokenID, probs, length, _ = beam
      output_text = tokenizer.decode(output_tokenID.squeeze(0).tolist())
      idx = clean_output(output_text, eos_token)
      output_text = ' '.join(output_text.split(' ')[:idx])
      probs = probs[:idx-1]

      score = cal_score(probs, len(probs), normalized=False)
      print(f"{output_text}  {round(np.exp(score), 2): .2f}")

In [ ]:
sample_seeds = ["to", "america", "people", "next", "picture", "on"]
max_gen_len=10
for seed in sample_seeds:
  gen_greedy(seed)
  gen_beam_False(seed)
  gen_beam_True(seed)
  print()

-Greedy-
to encourage creativity in the new york bill   0.13
-Unnormalized-
to consult exploring recipes for new jersey   0.00
to consult exploring recipes up the pacific northwest   0.00
-Normalized-
to consult exploring recipes up the pacific northwest   0.17
to consult exploring recipes up the least of the week   0.16

-Greedy-
america s lethal export   0.25
-Unnormalized-
america s lethal export   0.02
america s desert aisles   0.01
-Normalized-
america s lethal export   0.25
america s desert aisles   0.20

-Greedy-
people to balloon to make a criminal with a dog with a callous rival   0.14
-Unnormalized-
people to balloon for a criminal   0.00
people to balloon for a criminal with trump   0.00
-Normalized-
people to balloon for a criminal with a second fiddle   0.13
people to balloon for a criminal with trump   0.13

-Greedy-
next phenom english clubs 2 call another deal in the same arrivals   0.12
-Unnormalized-
next s blist revue   0.00
next phenom english clubs 1 a chance to be

Your output should be:


```
-Greedy-
to encourage creativity in the new york bill  0.13
-Unnormalized-
To Consult Exploring Recipes For New Jersey 0.00
To Consult Exploring Recipes Up The Pacific Northwest 0.00
-Normalized-
To Consult Exploring Recipes Up The Pacific Northwest 0.17
To Consult Exploring Recipes Up The Least Of The Week 0.16

-Greedy-
america s lethal export  0.25
-Unnormalized-
America S Lethal Export 0.02
America S Desert Aisles 0.01
-Normalized-
America S Lethal Export 0.25
America S Desert Aisles 0.20

-Greedy-
people to balloon to make a criminal with a dog with a callous rival  0.14
-Unnormalized-
People To Balloon For A Criminal 0.00
People To Balloon For A Criminal With Trump 0.00
-Normalized-
People To Balloon For A Criminal With A Second Fiddle 0.13
People To Balloon For A Criminal With Trump 0.13

-Greedy-
next phenom english clubs 2 call another deal in the same arrivals  0.12
-Unnormalized-
Next S Blist Revue 0.00
Next Phenom English Clubs 1 A Chance To Be Back 0.00
-Normalized-
Next S Blist Revue 0.14
Next Phenom English Clubs 1 A Chance To Be Back 0.14

-Greedy-
picture perfect chapter a spot of view of banning care  0.08
-Unnormalized-
Picture Perfect Use Coffee 0.00
Picture Korean A Bonanza Of Pancakes 0.00
-Normalized-
Picture Korean A Bonanza Of Contemplation Times Of Trump S Son 0.12
Picture Korean A Bonanza Of Pancakes 0.07

-Greedy-
on the catwalk in saudi arabia  0.19
-Unnormalized-
On The Billboard Chart 0.00
On The Catwalk In Saudi Arabia 0.00
-Normalized-
On The Whole30 Diet Vowing To Eat Smarter Carbs To Be 0.27
On The Whole30 Diet Vowing To Eat Smarter Carbs For Because 0.26

```



# Answer Questions in MyCourseVille!

Use the seed word "usa" to answer questions in MCV.

In [ ]:
max_gen_len=10
gen_greedy("usa")
gen_beam_False("usa")
gen_beam_True("usa")

-Greedy-
usa s duty to investigate   0.09
-Unnormalized-
usa s duty to investigate   0.00
usa s bleak season 3 episode 4 recap chicken   0.00
-Normalized-
usa s bleak season 1 episode 2 darkness descends   0.19
usa s bleak season 3 episode 4 recap a seduction   0.18
